In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

company_info = pd.read_csv("yahoo_company_info_orig.csv")
company_info = company_info.sort_values('marketCap', ascending=False).reset_index(drop=True)

# This should be removed once we recreate our initial csv
# yfinance growth metrics kinda suck?
remove_keys = ['pegRatio', 'overallRisk', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 
               'revenueGrowth', 'earningsGrowth', 'earningsQuarterlyGrowth']
company_info = company_info.drop(remove_keys, axis=1)

In [ ]:
company = yf.Ticker("MSFT")

# company_5quarter_metrics = pd.concat([company.quarterly_income_stmt, company.quarterly_balance_sheet, company.quarterly_cashflow], join='inner')
# company_5quarter_metrics.loc[['Capital Expenditure Reported', 'Operating Cash Flow']]
# for v in company_5quarter_metrics.index:
#     print(v)

In [ ]:
cs = []
for c in company.balance_sheet.index:
    if 'debt' in c.lower():
        cs.append(c)
company.balance_sheet.loc[cs]


In [ ]:
company.income_stmt.loc['Basic EPS'][0], company.income_stmt.loc['Basic EPS'][3]

In [ ]:
(company.income_stmt.loc['Basic EPS'][0]/company.income_stmt.loc['Basic EPS'][3])**(1/3) - 1

In [ ]:
company.quarterly_balance_sheet

In [ ]:
company.quarterly_balance_sheet

In [ ]:
company.quarterly_balance_sheet

In [ ]:
company.quarterly_income_stmt.loc['Basic EPS']

In [ ]:
company.quarterly_income_stmt.loc["Net Income"]

In [ ]:
company.info['earningsGrowth']

In [ ]:
company.quarterly_balance_sheet

In [ ]:
def safe_get(df, keys):
    if isinstance(keys, str):
        keys = [keys]
    for key in keys:
        try:
            return df.loc[key].iloc[0]
        except Exception:
            continue
    return None

info = company.info
metrics = {}

ttm_equity = safe_get(company.quarterly_balance_sheet, ["Total Stockholder Equity", "Total Equity"])

In [ ]:
ttm_equity

In [ ]:
# provides Ex-Dividend Dates and are not aligned with quarterly schedule
# Would need Annualized Dividend Per Share and divide by stock price
company.dividends 

In [ ]:
# Initial Filters
# 1. company: company nane can be NaN, these companies are insignificant
# 2. returnOnEquity: NaN means its either a newer/delisted company
# 3. returnOnAssets: NaN means its either a newer/delisted company
# 4. heldPercentInsiders: Appear to be smaller companies or very new
# There are lots of companies with
# - trailingPE attempts to answer "How many times are investors willing to pay for each dollar of the company's earnings?"
#   Therefore a negative EPS is meaningless for this question.
#   We should probably calculate this
# - forwardPE can also be NaN if expected future EPS if <= 0.
drop_na = ['company', 'returnOnEquity', 'returnOnAssets', 'marketCap',
           'heldPercentInsiders', 'heldPercentInstitutions', 'trailingPE']
company_info = company_info.dropna(subset=drop_na)

# yfinance sets dividendYield to NaN even though 0 makes perfect sense
company_info['dividendYield'] = company_info['dividendYield'].fillna(0)
company_info['fiveYearAvgDividendYield'] = company_info['fiveYearAvgDividendYield'].fillna(0)

# - freeCashflow can have a lot of NaNs ()
# - enterpriseToEbitda also has a lot of NaNs, Remove metric
drop_for_now = ['freeCashflow', 'enterpriseToEbitda', 'debtToEquity', # important to calculate but lots missing
                'beta', # somewhat important but ARM is somehow missing
                'forwardPE', 'payoutRatio']
company_info = company_info.dropna(subset=drop_for_now)

In [ ]:
company_info.columns

In [ ]:
for col in company_info.columns:
    percent_na = company_info[col].isna().sum() / len(company_info)
    print(f"{col} has {percent_na:0.5f} NaN rows")

In [ ]:
company_info[company_info['beta'].isna()]

In [ ]:
company_info.query('marketCap >= 1_000_000_000')